In [1]:
import pandas as pd

In [3]:
df_gcp = pd.read_csv("storage_pricing_gcp.csv")
df_network = pd.read_csv("network_cost_v2.csv")
df_storage = pd.read_csv("storage_pricing.csv")

In [4]:
# Patch in prices for: put, get, get transfer, put transfer

# Fixed prices per storage tier
fixed_prices = {
    "Standard Storage": {
        "put request": 0.005/1000,
        "get request": 0.0004/1000,
        "get transfer": 0.0,
        "put transfer": 0.0,
    },
    "Nearline Storage" : {
        "put request": 0.01/1000,
        "get request": 0.001/1000,
        "get transfer": 0.01,
        "put transfer": 0.01,
    },
    "Coldline Storage" : {
        "put request": 0.02/1000,
        "get request": 0.01/1000,
        "get transfer": 0.02,
        "put transfer": 0.02,
    },
}

df_gcp["Group"] = "storage"
# StartingRange,EndingRange
df_gcp["StartingRange"] = 0
df_gcp["EndingRange"] = float("inf")

# Patch in prices for: put, get, get transfer, put transfer
# Create new rows for fixed prices
new_records = []
for storage_tier in fixed_prices:
    for row in df_gcp[(df_gcp["Tier"] == storage_tier) & (df_gcp["Name"] == "gcs")].iterrows():
        #display(row[1])
        row = row[1]

        # Record for storage price from file
        new_records.append({
            "Vendor": row["Vendor"],
            "Name": row["Name"],
            "Region": row["Region"],
            "Group": "storage",
            "Tier": storage_tier,
            "Unit": float("NaN"),
            "StartingRange": 0,
            "EndingRange": float("inf"),
            "PricePerUnit": row["PricePerUnit"],
        })

        # Record for other prices from dict
        for price_type in fixed_prices[storage_tier]:
            new_records.append({
                "Vendor": row["Vendor"],
                "Name": row["Name"],
                "Region": row["Region"],
                "Group": price_type,
                "Tier": storage_tier,
                "Unit": float("NaN"),
                "StartingRange": 0,
                "EndingRange": float("inf"),
                "PricePerUnit": fixed_prices[storage_tier][price_type],
            })

df_new = pd.DataFrame(new_records)
df_storage_concat = pd.concat([df_storage, df_new], ignore_index=True)
df_storage_concat.to_csv("storage_pricing.csv", index=False)

In [13]:
storage_gcp_region = set(df_gcp["Region"].unique())
network_gcp_dest_region = set(df_network[df_network["dest_vendor"] == "gcp"]["dest_region"].unique())

missing_storage_gcp_region = storage_gcp_region - network_gcp_dest_region
missing_network_gcp_region = network_gcp_dest_region - storage_gcp_region
display(missing_storage_gcp_region)
display(missing_network_gcp_region)

set()

set()